In [1]:
# %pip install huggingface_hub
# huggingface-cli login (in terminal)
# %pip install protobuf
# %pip install sentencepiece

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch


print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             torch_dtype=torch.bfloat16, 
                                             device_map=device)



True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
file_id = 1
answer_df_path = "../../bloom_data/answer_df_part"+str(file_id)+".csv"

In [4]:
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data/fea_df.csv")

if not 'answer_string' in answer_df.columns:
    answer_df['answer_string'] = np.nan
    
print(answer_df.shape)
print(answer_df.iloc[(answer_df.shape[0]-10):answer_df.shape[0],[0,2]])

(9647, 3)
        sm_id answer_string
9637  177j4fc           NaN
9638  177k1er           NaN
9639  177myvb           NaN
9640  177nxm0           NaN
9641  177p7sh           NaN
9642  177po71           NaN
9643  177q3n4           NaN
9644  177qma8           NaN
9645  177qzcx           NaN
9646  177rle5           NaN


In [5]:
# # in replace of system message
# def get_topic_label(label: str, relatedphrases: str):
#     """
#     Answer whether any topic exists in a paragraph. Only return answer.
    
#     Args:
#         label: whether a topic exists in the given paragraph, (yes/no).
#         relatedphrases: if yes, provide the phrases in the paragraph that are related to the topic. (related phrases: ...)
#     """
#     pass
# in replace of system message
system_message = """
You are an AI assistant designed to answer questions.
Please restrict your answer to the exact question and use the exact answer format asked.
Answer should not have implied information. If the answer is yes, always provide related phrases. 
"""

def get_topic_label():
    """
    Answer questions.
    Restrict to the exact answer format.
    Provide related phrases from the paragraph, only if the answer is yes.
    """
    pass
tools = [get_topic_label]

def format_user_message(text):
    question_content = "Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format: Answer starts here:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: yes/no, related phrases if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = question_content + paragragh_content + answer_content
    #print(user_message)
    
    return user_message




In [6]:
# from tqdm import tqdm

# for k in tqdm(range(1), desc="Processing batch"):
    
#     batch_size = 10

#     # Filter for rows where 'answer_string' is NaN
#     unanswered_df = answer_df[answer_df['answer_string'].isna()]

#     # Get the indices of these NaN entries in the original DataFrame
#     indices_to_update = unanswered_df.index[:batch_size]

#     # Prepare prompt content for the first 10 entries with NaN answer_string
#     user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

#     # Save the indices list if needed for later use
#     indices_to_update_list = list(indices_to_update)


#     if len(indices_to_update_list)>0:
#         for i in range(len(user_messages)):
#             failed = True
#             fail_counter = 0
#             while failed and fail_counter<10:
#                 conversation = [
#                     {"role": "system", "content": system_message},
#                     {"role": "user", "content": user_messages[i]}
#                 ]
#                 inputs = tokenizer.apply_chat_template(
#                         conversation,
#                         # tools=tools,
#                         add_generation_prompt=False,
#                         return_dict=True,
#                         return_tensors="pt")
#                 inputs.to(model.device)
#                 outputs = model.generate(**inputs,
#                                          do_sample=True,
#                                          max_new_tokens=512,
#                                          temperature=0.1)
#                 response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#                 del inputs
                
#                 if "Answer starts here:" in response:
#                     answer = response.split("Answer starts here:")[-1]
#                     # Use the original index from indices_to_update_list
#                     answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer
#                     print(answer)
#                     failed = False
#                 else:
#                     failed = True
#                     fail_counter += 1
#                     # Use the original index from indices_to_update_list
#                     answer_df.loc[indices_to_update_list[i], 'answer_string'] = "Answer not found"

#         print(answer_df.loc[indices_to_update_list, ['sm_id','answer_string']])
#         answer_df.to_csv(answer_df_path, index=False)
#     else:
#         break
    
    

In [ ]:
from tqdm import tqdm

for k in tqdm(range(1000), desc="Processing batch"):
    
    batch_size = 10

    # Filter for rows where 'answer_string' is NaN
    unanswered_df = answer_df[answer_df['answer_string'].isna()]

    # Get the indices of these NaN entries in the original DataFrame
    indices_to_update = unanswered_df.index[:batch_size]

    # Prepare prompt content for the first 10 entries with NaN answer_string
    user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

    # Save the indices list if needed for later use
    indices_to_update_list = list(indices_to_update)


    if len(indices_to_update_list)>0:
        for i in range(len(user_messages)):
            failed = True
            failed_count = 0
            while failed and failed_count<10:
                conversation = [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_messages[i]}
                ]
                inputs = tokenizer.apply_chat_template(
                        conversation,
                        # tools=tools,
                        add_generation_prompt=False,
                        return_dict=True,
                        return_tensors="pt")
                inputs.to(model.device)
                outputs = model.generate(**inputs,
                                         do_sample=True,
                                         max_new_tokens=512,
                                         temperature=0.1)
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                del inputs
                if "Answer starts here:" in response:
                    answer = response.split("Answer starts here:")[-1]
                    # Use the original index from indices_to_update_list
                    answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer
                    # print(answer)
                    failed = False
                else:
                    failed = True
                    failed_count += 1
                    # Use the original index from indices_to_update_list
                    answer_df.loc[indices_to_update_list[i], 'answer_string'] = "Answer not found"

        print(answer_df.loc[indices_to_update_list, ['sm_id','answer_string']])
        answer_df.to_csv(answer_df_path, index=False)
    else:
         break
    

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   0%|          | 1/1000 [01:45<29:16:34, 105.50s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        sm_id                                      answer_string
7775  165hrc7  \n  (1) relation: No, related phrases if any: ...
7776  165ifgh  \n  (1) relation: No related phrases.\n  (2) p...
7777  165ijf5  \n  (1) relation: No related phrases.\n  (2) p...
7778  165kyet  \n  (1) relation: yes, related phrases: discus...
7779  165l4f8  \n  (1) relation: No related phrases.\n  (2) p...
7780  165lkz1  \n  (1) relation: Not mentioned.\n  (2) protei...
7781  165loz8  \n  (1) relation: No related phrases\n  (2) pr...
7782  165maqh  \n  (1) relation: yes, related phrases: brothe...
7783  165o0mu  \n  (1) relation: yes, related phrases: orderi...
7784  165oa18  \n  (1) relation: no\n  (2) protein: no\n  (3)...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   0%|          | 2/1000 [03:45<31:39:46, 114.22s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        sm_id                                      answer_string
7785  165qs6a  \n  (1) relation: yes, related phrases: 'i cou...
7786  165r2r2  \n  (1) relation: No, related phrases if any: ...
7787  165r4a7  \n  (1) relation: No, related phrases if any: ...
7788  165rjmn  \n  (1) relation: yes, related phrases: social...
7789  165rrpv  \n  (1) relation: Yes, related phrases: "partn...
7790  165suww  \n  (1) relation: No, related phrases if any: ...
7791  165t1xf  \n  (1) relation: yes, related phrases: meal p...
7792  165t3k3  \n  (1) relation: No, related phrases if any: ...
7793  165vx9l  \n  (1) relation: yes, related phrases: "I'm a...
7794  165w4bw  \n  (1) relation: No related phrases.\n  (2) p...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   0%|          | 3/1000 [05:33<30:46:00, 111.09s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   6%|▋         | 64/1000 [1:55:46<27:40:19, 106.43s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        sm_id                                      answer_string
8405  16jvaom  \n  (1) relation: No, related phrases if any: ...
8406  16jvbfi  \n  (1) relation: No, the paragraph does not m...
8407  16jvtdf  \n  (1) relation: No, related phrases if any: ...
8408  16jx8e5  \n  (1) relation: yes, related phrases: native...
8409  16jxadd  \n  (1) relation: yes, related phrases: "losin...
8410  16jxujf  \n  (1) relation: yes, related phrases: diet\n...
8411  16jzmyo  \n  (1) relation: yes, related phrases: recipe...
8412  16k06yl  \n  (1) relation: No related phrases.\n  (2) p...
8413  16k28je  \n  (1) relation: no\n  (2) protein: no\n  (3)...
8414  16k2sxk  \n  (1) relation: yes, related phrases: husban...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batch:   6%|▋         | 65/1000 [1:57:29<27:23:49, 105.49s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


        sm_id                                      answer_string
8415  16k4euw  \n  (1) relation: no, related phrases if any: ...
8416  16k4kxj  \n  (1) relation: yes, related phrases: wife\n...
8417  16k4mly  \n  (1) relation: No, related phrases if any: ...
8418  16k7jkq  \n  (1) relation: yes, related phrases: wife, ...
8419  16k83q3  \n  (1) relation: yes, related phrases: family...
8420  16k9fvr  \n  (1) relation: yes, related phrases: lookin...
8421  16kc5q7  \n  (1) relation: No related phrases.\n  (2) p...
8422  16kcth9  \n  (1) relation: No related phrases.\n  (2) p...
8423  16kdidr  \n  (1) relation: yes, related phrases: body r...
8424  16keffw  \n  (1) relation: Not mentioned\n  (2) protein...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
